# EHR

*Classes*
   - Patient
     - CrCl Calculation Display
   - LabReference

In [1]:
import sympy as sp
import random
from IPython.display import Markdown
import pandas as pd

## Patient (class)

   - CrCl (function)

In [2]:
%%writefile PatientClass.py
import sympy as sp
import random
from IPython.display import Markdown
import pandas as pd

class Patient:
   """### Class: PATIENT
   ----------------------------------
   *INITS*
   >>> name
   >>> weight
   >>> age
   >>> SerCreat
   """
   def __init__(self, name, weight, age, SerCreat, female:bool):
      """### Class: PATIENT
      ***
      """
      self.gendcode = 1
      if female == True:
         self.gendcode = 0.85
      self.name =       name
      self.id =         random.randint(1000000,10000000)
      self.weight =     weight
      self.age =        age
      self.SerCreat =   SerCreat
      self.CrCl = ((140-self.age) * self.weight) / (72 * self.SerCreat)
   
   def as_dict (self):
      return {
         "name":              self.name, 
         "id":                self.id, 
         "weight":            self.weight, 
         "age":               self.age, 
         "SerumCreatinine":   self.SerCreat, 
         "CrCl":              self.CrCl
         }
   
   @property
   def CrCl_show (self):
      name =      self.name
      age =       self.age 
      weight =    self.weight
      scr =       self.SerCreat
      
      a , w , Cr = sp.symbols("a_yr W_kg Cr_Ser")
      CrCl = ((140-a) * w) / (72 * Cr)
      
      # Dependent Var = 
      md1 = "$\large CrCl_{" + name + "} = $" 
      # View Input Subs
      md2 = "$ " + sp.Subs(CrCl,(a,w,Cr),(age,weight,scr))._repr_latex_()[1:] + "$"
      # Solution
      md3 = "$\large = \\boxed {" + str(round(CrCl.subs({a:age,w:weight,Cr:scr}),1)) + "}$"
      
      table = Markdown (f"Solve for | Subs | Solution\n----|----|----\n{md1} | {md2} | {md3}\n")
      
      return table

   def __repr__(self):
      return f'Name: {self.name}\n\tID:{self.id}\n\t{self.weight}kg\n\t{self.age}yr.of age\n\n\tCr(serum): {self.SerCreat}\n\t{self.CrCl}'

Overwriting PatientClass.py


In [3]:
import PatientClass
from PatientClass import Patient

In [4]:
Josh = Patient("Josh Steinbecker",68,27, 0.988,False)
Josh.CrCl
Josh.CrCl_show

Solve for | Subs | Solution
----|----|----
$\large CrCl_{Josh Steinbecker} = $ | $ \displaystyle \left. \frac{W_{kg} \left(140 - a_{yr}\right)}{72 Cr_{Ser}} \right|_{\substack{ a_{yr}=27\\ W_{kg}=68\\ Cr_{Ser}=0.988 }}$$ | $\large = \boxed {108.000000000000}$


In [5]:
Patient("Bob", 79, 59, 1.629,False).CrCl_show



Solve for | Subs | Solution
----|----|----
$\large CrCl_{Bob} = $ | $ \displaystyle \left. \frac{W_{kg} \left(140 - a_{yr}\right)}{72 Cr_{Ser}} \right|_{\substack{ a_{yr}=59\\ W_{kg}=79\\ Cr_{Ser}=1.629 }}$$ | $\large = \boxed {54.6}$


## LabReference (class)
```py
   >>> refID
   >>> Name
   >>> Low
   >>> High
   >>> Unit *
   >>> Class *
```

In [21]:
class LabReference:
   """
   ### LAB REFERENCE
   #### Inits :
   >>> refID
   >>> Name
   >>> Low
   >>> High
   ------------
   >>> Unit*      default= None
   >>> Class*     default= "Chem"
   """
   refID:      str
   name:       str
   lowbound:   float
   highbound:  float
   unit:       str
   
   def __init__(self,refID,name,low,high,Unit=None,Class="Chem"):
      self.refID = refID
      self.Class = Class
      self.name = name
      self.lowbound = low
      self.highbound = high
      self.unit = Unit
      
   @property
   def therIndex (self):
      avgVal = round((self.lowbound + self.highbound) / 2,1)
      dev = avgVal - self.lowbound
      ti = dev / avgVal
      return f'{ti:.0%}'
   def __repr__(self):
      return f'[{self.name} ({self.lowbound} - {self.highbound})]'
   def as_dict (self):
      return {"id": self.refID, "class":self.Class,"name":self.name, "low": self.lowbound, "high": self.highbound, "unit":self.unit, "therIndex":self.therIndex}

In [29]:
NA = LabReference          ("NA","Sodium(serum)",136,142,"mg/dL")
K = LabReference           ("K","Potassium(serum)",3.5,5.1,"mg/dL")
CA = LabReference          ("CA","Calcium(serum)",8.1,10,"mg/dL")
HCO3 = LabReference        ("HCO3","Bicarbonate",21,28,"mg/dL")
GLU = LabReference         ("GLU","Glucose",90,100,"mg/dL")
VANC_TROUGH = LabReference ("VANCO[TROUGH]","Vancomycin  (serum/trough)",11,20,"mg/dL","ABX")
PLT = LabReference         ("PLT","Platelet",150000,400000,"/mL","Hemo")

ALLLABS = [NA,K,CA,HCO3,VANC_TROUGH,GLU,PLT]
ALLLABS


[[Sodium(serum) (136 - 142)],
 [Potassium(serum) (3.5 - 5.1)],
 [Calcium(serum) (8.1 - 10)],
 [Bicarbonate (21 - 28)],
 [Vancomycin  (serum/trough) (11 - 20)],
 [Glucose (90 - 100)],
 [Platelet (150000 - 400000)]]

In [30]:

df = pd.DataFrame([x.as_dict() for x in ALLLABS])  # Create Table
df.set_index(["class","id"]).sort_index()          # Multindex by 1:Class then 2:ID

name       low      high   unit  \
class id                                                                     
ABX   VANCO[TROUGH]  Vancomycin  (serum/trough)      11.0      20.0  mg/dL   
Chem  CA                         Calcium(serum)       8.1      10.0  mg/dL   
      GLU                               Glucose      90.0     100.0  mg/dL   
      HCO3                          Bicarbonate      21.0      28.0  mg/dL   
      K                        Potassium(serum)       3.5       5.1  mg/dL   
      NA                          Sodium(serum)     136.0     142.0  mg/dL   
Hemo  PLT                              Platelet  150000.0  400000.0    /mL   

                    therIndex  
class id                       
ABX   VANCO[TROUGH]       29%  
Chem  CA                  11%  
      GLU                  5%  
      HCO3                14%  
      K                   19%  
      NA                   2%  
Hemo  PLT                 45%